# Multiclass classification using _ImaGene_

In this exercise you will learn how to perform a multiclass classification using _ImaGene_.
In this example, your goal is to classify whether a given _locus_ is under positive selection or neutrally evolving with 3 classes of selection coefficient (0,200,400 in $2N_e$ units with $N_e=10000$).

You will be using the case of lactase persistence in European populations following the data and example of `01_binary` notebook.
Please refer to the tutorial for binary classification for an in-depth explanation of the case study and help on the main steps of the pipeline.
Here you will also learn the main differences with the case of binary classification.
Your job is to fill in lines with `# ...`.

In [ ]:
import os
import gzip
import _pickle as pickle

import numpy as np
import scipy.stats
import pymc3

import tensorflow as tf
from tensorflow import keras
from keras import models, layers, activations, optimizers, regularizers
from keras.utils.vis_utils import plot_model
from keras.models import load_model

import itertools
import skimage.transform
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pydot # optional, but required by keras to plot the model

In [ ]:
%run -i ../ImaGene.py

### 1. Read data from VCF file and store it into _ImaGene_ objects

In [ ]:
file_LCT = ImaFile(nr_samples=198, VCF_file_name='LCT.CEU.vcf')
gene_LCT = file_LCT.read_VCF()

Here we assume we are unsure about the ancestral/derived polarisation and we switch labelling of alleles to major/minor.
We also then retain all polymorphic sites and discard columns corresponding to sites with a sample minor allele frequency of 0 (e.g., a suitable threshold is 0.005 which is < 1/198).

In [ ]:
# perform the data processing explained above
# ...
# ...

Check the dimensions and plot it.

In [ ]:
gene_LCT.summary();
gene_LCT.plot();

As an illustration, we order rows based on their genetic distance from the most frequent haplotype. 

In [ ]:
# ... hint: check help page gene_LCT.sort?
gene_LCT.plot();

### 2. Run and process simulations to be used for training the neural network

We provide an example of parameter file called `params_multiclass` which simulates a total of 250,000 loci of 80kbp either under neutral evolution or positive selection with additive effect and allelic selection coefficients of 100, 200, 300, or 400 in $2N_e$ units with $N_e=10,000$.
All other parameters are set as in the example of binary classification.

In [ ]:
# set as appropriate, e.g.:
# path_sim='/home/mfumagal/Data/ImaGene/Tutorials/' # my local machine
# path_sim='/mnt/quobyte/ImaGene/' # for workshop spp1819
path_sim = './'

Open, edit, save and class `params_multiclass.txt` file to specify directories and simulation parameters. Here I assume that simulations will be stored in `path+Multiclass/`.

In [ ]:
# if you wish to generate new training data, do not run otherwise
import subprocess
subprocess.call("bash ../generate_dataset.sh params_multiclass.txt".split())

As an illustration, read the first batch of simulations to explore their content. Retain only 200 data points per class for a quick example (or even less).

In [ ]:
# fill in the missing value ...
file_sim = ImaFile(simulations_folder=path_sim + 'Multiclass/Simulations1', nr_samples=198, model_name='Marth-3epoch-CEU');
gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=...);

How many images do you expect?

In [ ]:
gene_sim.summary();

As expected we have 1000 images (if you used 200 for each one of the 5 classes).
The average number of columns is $\approx ...$

We can check the sample allele frequency for the selected allele. Recall that we imposed selection to be acting in the middle of the region. Therefore, the targeted allele will be in position '0.5' in the _msms_ file.

In [ ]:
freqs = calculate_allele_frequency(gene_sim, 0.5);
plt.scatter(gene_sim.targets, freqs, marker='o');
plt.xlabel('Selection coefficient');
plt.ylabel('Allele frequency');

Perform the same data processing as done for LCT. As an example, we resize all images to the average dimensions of (198,285).

In [ ]:
# you need to switch to major/minor, filter based on allele frequencies, sort rows and resize
# ...
# remember to add `gene_sim.convert();` at the end

Suppose that we are just interested in a classification of 3 classes: 0, 200, 400.
For doing that, first we need to set `.classes` to the desired values and then we need to take a subset of the data corresponding to the desired classes only.
We can achieve these steps with the following lines.

In [ ]:
gene_sim.classes = np.array([0,200,400]);
gene_sim.subset(get_index_classes(gene_sim.targets, gene_sim.classes));
gene_sim.summary();

How many data points you have?

We now have 600 data points corresponding to the desired classes, as expected. 

### 3. Implement, train and evaluate the neural network

As discussed in the tutorial on binary classification, we need to repeat this procedure across all batches of data leaving the last one for testing. We retain 1000 images per class as an illustration.

In [ ]:
# fill in all missing values (...) or lines
i = 1
while i <= 10:

    # simluations 
    file_sim = ImaFile(simulations_folder=path_sim+'Multiclass/Simulations' + str(i), nr_samples=198, model_name='Marth-3epoch-CEU')

    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=...)
    
    # manipulate data:
    # switch to major/minor allele polarisation
    # filter our monomorphic sites
    # sort rows by genetic distance
    # ...
    # ...
    # ...
    # ...
    gene_sim.convert()
    
    # use only classes 0,200,400
    gene_sim.classes = np.array([0,200,400])
    gene_sim.subset(get_index_classes(gene_sim.targets, gene_sim.classes))

    # randomise data
    gene_sim.subset(get_index_random(gene_sim))

    # convert targets to categorical data
    gene_sim.targets = to_categorical(gene_sim.targets)
    
    # at first iteration we build the model 
    if i == 1:

        model = models.Sequential([ # build your network! you can use a similar architecture to the binary case
                    # ...
                    # ...
                    layers.Flatten(),
                    layers.Dense(units=128, activation='relu'),
                    # for the last dense layer, how many output units do you have? also, what is the activation function?
                    layers.Dense(units=..., activation=...)])
        model.compile(optimizer='adam',
                    loss='...', # which loss do you need? check https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
                    metrics=['accuracy'])

        net_LCT = ImaNet(name='[C32+P]+[C64+P]x2+D128') # name is not really important

    # training for iterations from 1 to 9
    if i < 10:
        score = model.fit(gene_sim.data, gene_sim.targets, batch_size=32, epochs=1, verbose=0, validation_split=0.10)
        net_LCT.update_scores(score)
    else:
        # testing for iteration 10
        net_LCT.test = ... 
        net_LCT.predict(...)

    i += 1

In [ ]:
# set working directory where to save models, e.g. 
# path='/home/mfumagal/Data/ImaGene/Tutorials/Data/' # my local machine
# path='./' # for workshop spp1819
path = './'

In [ ]:
# just as an illustration....
# save final (trained) model
model.save(path+'model.multi.h5');

# save testing data
gene_sim.save(path+'gene_sim.multi');

# save network
net_LCT.save(path+'net_LCT.multi');

In [ ]:
# just as an illustration....
# load the objects above
model = load_model(path+'model.multi.h5');
gene_sim = load_imagene(file=path+'gene_sim.multi');
net_LCT = load_imanet(path+'net_LCT.multi');

In [ ]:
# assess the training, plot loss and accuracy over epochs
# ...

In [ ]:
# print the testing results [loss, accuracy]
print(net_LCT.test);

In [ ]:
# plot a confusion matrix (on the last gene_sim object which represents the testing data)
net_LCT.#...

What conclusions can you make from the assessment of the training?

### 4. Deploy the trained network on your genomic data of interest

Finally we use the trained network to predict the class of natural selection on our locus of interest.
First we need to resize our real data to match the data used for training.
After that, we need to convert the data into the required format.

In [ ]:
# ... resize...
gene_LCT.convert();
gene_LCT.summary();

In [ ]:
print(model.predict(gene_LCT.data, batch_size=None));

What is the output of the command above?  What do these values mean?